#Regressão Logística com Python

Iremos trabalhar com o [Titanic Data Set from Kaggle](https://www.kaggle.com/c/titanic). Este é um conjunto de dados muito famoso e muitas vezes é o primeiro passo de um aluno no aprendizado de máquina!

Tentaremos prever uma classificação: sobrevivência ou falecimento.
Vamos começar a compreensão da implementação da regressão logística em Python para classificação.

Usaremos uma versão "semi-limpa" do conjunto de dados do Titanic, se você usar o conjunto de dados hospedado diretamente no Kaggle, talvez seja necessário fazer algumas limpezas adicionais.

##Importar Bibliotecas

Vamos importar algumas bibliotecas para começar!

##Os Dados


Vamos começar lendo o arquivo titanic_train.csv em um pandas dataframe.

In [ ]:
# Esse trecho do código é obrigatório para quem estiver fazendo tudo pelo colab
# Caso você esteja utilizando o jupyter pode comentar/apagar
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir("drive/My Drive/Colab Notebooks/13_Regressão_Logística")
os.listdir()

#Exploratory Data Analysis

Vamos começar uma análise exploratória de dados! Começaremos verificando os dados ausentes!

##Missing Data

Podemos usar o seaborn para criar um mapa de calor simples para ver onde estão faltando dados!

Cerca de 20 por cento dos dados de idade estão ausentes. A proporção de Idade ausente é, provavelmente, pequena o suficiente para que uma substituição com alguma forma de inputação fique razoável. Olhando para a coluna Cabine, no entanto, parece que um número excessivo destes dados estão faltando para se obter alguma informação útil com uma análise básica. Provavelmente vamos abandonar essa coluna ou alterá-la para uma outra característica como "Cabin Known: 1 or 0"

Vamos continuar visualizando mais alguns dados!

##Data Cleaning

Queremos preencher os dados de idade ausentes em vez de apenas descartar as linhas de dados de idade ausentes. Uma forma de fazer isso é preenchendo com a idade média de todos os passageiros (imputação).

No entanto, podemos ser mais espertos quanto a isso e verificar a idade média por classe de passageiros. Por exemplo:


Podemos ver que os passageiros nas classes mais altas (mais ricos) tendem a ser mais velhos. Usaremos esses valores de idade média para imputar com base em Pclass por Age.

Aplicando essa função!

Verificando o mapa de calor novamente!

Excelente! Vamos em frente e remover a coluna Cabin e a linha em Embarked que é NaN.

##Convertendo Características Categóricas 

Precisaremos converter características categóricass em variáveis ​​fictícias usando pandas! Caso contrário, nosso algoritmo de aprendizado de máquina não poderá receber diretamente essas características como entradas.

Excelente! Nossos dados estão prontos para o nosso modelo!

#Construindo um Modelo de Regressão Logística

Vamos começar dividindo nossos dados em um conjunto de treinamento e um conjunto de teste (há outro arquivo test.csv com o qual você pode brincar caso queira usar todos esses dados para treinamento).

##Train Test Split

##Treinamento e Prevendo

Vamos avaliar o modelo!

##Avaliação

Podemos verificar precision, recall, f1-score utilizando classification_report!

Não ficou tão ruim! Você pode explorar outras características e o outro arquivo titanic_test.csv, algumas sugestões:

* Tente utilizar o título (Dr., Mr., Mrs, etc.) do nome como uma característica
* Talvez a letra Cabin possa ser uma característica
* Existe alguma informação que você pode obter do bilhete (ticket)?